# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.txt** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **1. clean all outputs and 2. run all cells in your notebook and keep all running logs** so that we can check.

# 1 $n$-gram Language Model

In [ ]:
!mkdir -p data/lm
!wget -O data/lm/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/train.txt
!wget -O data/lm/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/dev.txt
!wget -O data/lm/test.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/test.txt

## 1.1 Building vocabulary

### Code

In [52]:

"""
description: Building Vocabulary
"""

from collections import defaultdict
import numpy as np

V_uni = defaultdict(int) # set default number 0 to all tokens
V_bi = defaultdict(int) # set default number 0 to all tokens
N_train = 0 # number of tokens
num_sentences = 0

def preprocess(vo_dict, wh_filter=1):
    voca_list = list(vo_dict.items())
    # TODO: use np.where to implement filter, instead of for
    sum_unk = 0
    if wh_filter == 1:
        voca_key, voca_values = zip(*voca_list)
        index = np.where(np.array(voca_values)>=3)
        valid_list = list(np.array(voca_list)[index])

        invalid_index = np.where(np.array(voca_values)<3)
        invalid_list = list(np.array(voca_list)[invalid_index])
        invalid_list = [int(i[1]) for i in invalid_list]
        sum_unk = sum(invalid_list)
    else:
        valid_list = voca_list
    valid_list.append(('<UNK>', sum_unk))
    valid_list.append(('<START>', num_sentences)) #【可以改成句子个数】
    valid_list.append(('<END>', num_sentences))
    return valid_list


with open("./data/lm/train.txt",'r+',encoding='utf-8') as f:
    for line in f.readlines():
        curline = line.strip().split(" ") # use function:strip to drop "\n"
        for item in curline: # 1.compute c(w)
            V_uni[item] += 1
        num_sentences += 1
# c(w) -- convert Vocabulary into different formats(list\ndarray\dict)
uni_list = preprocess(V_uni, wh_filter=1)
uni_array = np.array(uni_list)
uni_dict = dict(uni_list)
values = uni_dict.values()
N_train = sum([int(i) for i in values ]) 
print("vocabulary size is",len(uni_list))



with open("./data/lm/train.txt",'r+',encoding='utf-8') as f:
    for line in f.readlines():
        curline = line.strip().split(" ") # use function:strip to drop "\n"
        for index in range(0, len(curline)):# 2.compute c(w,u)
            if curline[index] not in uni_dict.keys(): # should be unk
                curline[index] = '<UNK>'

            if index == 0:
                string = curline[index]+"|"+'<START>'
            elif index != len(curline)-1:
                if curline[index+1] not in uni_dict.keys(): # should be unk
                    curline[index+1] = '<UNK>'
                string = curline[index+1] + "|" + curline[index] 
                # contaminate two string, with shape "new|old"
            else:
                string = '<END>'+ '|' + curline[index] # w=<END>

            V_bi[string] += 1

# c(w,u) -- adjacent tokens pair
bi_list = preprocess(V_bi, wh_filter=0)
bi_array = np.array(bi_list)
bi_dict = dict(bi_list)





vocabulary size is 26601


### Discussion：
discuss the number of parameters of n-gram models.

TBD

## 1.2 $n$-gram Language Modeling

### Code

In [66]:

"""
description: build bigram and unigram language models and
report their perplexity on the training set, and dev set
"""
import copy
import math


class N_Gram():
    def __init__(self, indicator, smoothing=None, k=None) -> None:
        self.indicator = indicator # which model: unigram or bigram
        self.smoothing = smoothing # which smoothing method to be selected
        self.k = k # add-k smoothing
        self.uni_model = None
        self.bi_model = None

    def train(self):
        if self.smoothing == None:
            return self.no_smoothing() # return a dictionary of unigram or bigram 
        elif self.smoothing == "add-one":
            pass
        elif self.smoothing == "add-k":
            pass
        
    def no_smoothing(self):

        if self.indicator == "unigram": # model: {token: probability}
            self.uni_model = copy.deepcopy(uni_dict) # frequency
            for (token, fre) in uni_dict.items():
                pro = int(fre) / N_train
                self.uni_model[token] = pro # "probability"
            return self.uni_model
            
        elif self.indicator == "bigram": # model: {token+1|token: probability}
            self.bi_model = copy.deepcopy(bi_dict)
            for (token, fre) in bi_dict.items():
                sep_index = token.find('|')
                last_token = token[sep_index+1 : ]
                
                if fre == 0: # c(w,u)=0
                    pro = 0
                elif last_token not in uni_dict.keys(): # frenquency<3
                    last_token = '<UNK>'
                    pro = int(fre) / int(uni_dict[last_token])
                else:
                    pro = int(fre) / int(uni_dict[last_token])
                self.bi_model[token] = pro # "probability"
            return self.bi_model
            
    def Perplexity(self,file="train"):
        if file == "train":
            path = "./data/lm/train.txt"
        elif file == "dev":
            path = "./data/lm/dev.txt"
        
        Likelihood = 0
        with open(path,"r+",encoding="utf-8") as f:
            M = 0
            for line in f.readlines():
                curline = line.strip().split(" ") 
                multiply = 1
                for index in range(0,len(curline)):
                    M += len(curline)
                    if self.indicator == "unigram":
                        if curline[index] not in uni_dict.keys():
                           curline[index] = '<UNK>' 
                        # 【此时，unigram永远不会是0！！！】
                        # 【如果验证集、测试集中，遇到了没有遇到过的token（不是frequency<3）应如何处理】
                        # 【feasible：记录一个UNK真实的列表，如果验证集的在，就用频率。不在，就设成0】
                        multiply *= uni_dict[curline[index]]

                    elif self.indicator == "bigram":
                        # preprocess: convert to UNK
                        if curline[index] not in uni_dict.keys():
                            curline[index] = '<UNK>'
                        # generate string
                        if index == 0:
                            string = curline[index]+'|'+'<START>'                    
                        elif index == len(curline)-1:
                            string = '<END>'+'|'+curline[index]
                        else:
                            if curline[index+1] not in uni_dict.keys():
                                curline[index+1] = '<UNK>'
                            string = curline[index+1]+'|'+curline[index]

                        if string not in self.bi_model.keys(): multiply = 0; break
                        else: multiply *= self.bi_model[string]
                if multiply == 0: # exists unseen bi-pairs
                    continue
                Likelihood += math.log(multiply) 
            Likelihood *= 1/M
        return 2**(-Likelihood) 
            
"""
test_1 = N_Gram(indicator='unigram', smoothing=None)
uni_model = test_1.train()
for i in range(200):
    print(list(uni_model.items())[i])
"""
test_1 = N_Gram(indicator='bigram', smoothing=None)
uni_model = test_1.train()
ple = test_1.Perplexity(file="dev")
print(ple)


1.0106502804332709


### Discussion： 
If you encounter any problems, please analyze them and explain why.

1. 由于删除了部分token（频率<3），部分算出来概率>1

## 1.3 Smoothing

### 1.3.1 Add-one (Laplace) smoothing

### Code

#### Discussion

#### 1.3.2 Add-k smoothing

##### Code

##### Discussion

### 1.3.3 Linear Interpolation

#### Code

#### Discussion

##### 1.3.4 Optimization

#### Discussion

# 2 Word Vectors

In [67]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 50
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin

wv_from_bin = load_embedding_model()

ModuleNotFoundError: No module named 'gensim'

## 2.1 Find most similar word
Use cosine similarity to find the most similar word to each of these words. Report the most similar word and its cosine similarity.

## 2.2 Finding Analogies
Use vector addition and subtraction to compute target vectors for the analogies below. After computing each target vector, find the top three candidates by cosine similarity. Report the candidates and their similarities to the target vector.

- dog : puppy :: cat : ?:
- speak : speaker :: sing : ?:
- France : French :: England : ?:
- France : wine :: England : ?

# 3 Sentiment analysis

In [ ]:
!mkdir -p data/classification
!wget -O data/classification/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/train.txt
!wget -O data/classification/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/dev.txt
!wget -O data/classification/test-blind.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/test-blind.txt

## 3.1 Logistic Regression

### Unigram Features

### Bigram Features

### GloVe Features

Compare the performance of three types of features on dev set. Report the weighted average precision, recall and F1-score for each feature set.

| Feature | precision | recall | F1-score |
| ----------- | --------- | ------ | -------- |
| unigram     |           |        |          |
| bigram      |           |        |          |
| GloVe       |           |        |          |

## 3.2 Better Feature